In [1]:
from keras.models import load_model
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
from glob import glob
import generator as gen
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
import tensorflow as tf
#from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet50

def dice_coeff(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coeff_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

def dice_metric(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

data_path_test = "G:\\Harsh_Birva_minor_Proj\\FIVES\\test"

retina_files_test = glob(pathname = data_path_test+'\\Original\\*')
retina_files_test[:1]

mask_files_test = [path.replace('Original', 'Ground truth') for path in retina_files_test]
mask_files_test[:1]

df_test = pd.DataFrame( data = {
    'retina': retina_files_test,
    'mask': mask_files_test
})

C:\Users\Admin\.conda\envs\tf\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
class ConvBlock(Layer):
    
    def __init__(self, filters=256//4, kernel_size=3, use_bias=False, dilation_rate=1, **kwargs):
        super(ConvBlock, self).__init__(**kwargs)
        
        self.filters = filters
        self.kernel_size = kernel_size
        self.use_bias = use_bias
        self.dilation_rate = dilation_rate
        
        self.net = Sequential([
            Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same', dilation_rate=dilation_rate, use_bias=use_bias, kernel_initializer='he_normal'),
            BatchNormalization(),
            ReLU()
        ])
    
    def call(self, X): return self.net(X)        
        
    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "filters":self.filters,
            "kernel_size":self.kernel_size,
            "use_bias":self.use_bias,
            "dilation_rate":self.dilation_rate
        }

In [3]:
def load_and_preprocess_image(file_path,resize):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (resize,resize))
    img_scaled = img_resized.astype(np.float32) / 255.0
    return img_scaled

In [4]:
def load_and_preprocess_mask(file_path,resize):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE )
    img = img[:,:,np.newaxis]
    img_resized = cv2.resize(img, (resize,resize))
    img_scaled = img_resized.astype(np.float32) / 255.0
    img_scaled[img_scaled >= 0.25] = 1
    img_scaled[img_scaled < 0.25] = 0
    return img_scaled

In [5]:
def DICE_COE(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = np.mean(dice)
    dice = round(dice, 3)
    return dice    

In [6]:
kernel = np.array([[0,1,0],[1,1,1],[0,1,0]],dtype=np.uint8)

In [7]:
def evaluate(model,resize,full=None,thin=None,thick=None):
    if full is None:
        full = []
    if thin is None:
        thin = []
    if thick is None:
        thick = []
        
    for i in range(200):
        retina_img = load_and_preprocess_image(df_test.iloc[i]['retina'],resize)
        mask_img = load_and_preprocess_mask(df_test.iloc[i]['mask'],resize)
        
        retina_img = retina_img[np.newaxis,:,:,:]
        pred_img = model.predict(retina_img)
        pred_img = (np.squeeze(pred_img) > 0.5).astype(np.uint8)
        mask_img = np.squeeze(mask_img).astype(np.uint8)

        full.append(DICE_COE(mask_img,pred_img))

        original = mask_img.copy()
        original_pred = pred_img.copy()

        pred_img = pred_img.astype(float)
        mask_img = mask_img.astype(float)

        for i in range(5):
            mask_img = cv2.erode(mask_img, kernel)
            pred_img = cv2.erode(pred_img, kernel)

            mask_img = cv2.dilate(mask_img, kernel)
            pred_img = cv2.dilate(pred_img, kernel)

        pred_img = pred_img.astype(np.uint8)
        mask_img = mask_img.astype(np.uint8)

        original_thin = cv2.subtract(original,mask_img).astype(np.uint8)
        predicted_thin = cv2.subtract(original_pred,pred_img).astype(np.uint8)

        thin.append(DICE_COE(original_thin,predicted_thin))
        thick.append(DICE_COE(mask_img, pred_img))
        
    return full,thin,thick

# My-Net 16 (16 blocks)

In [117]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [118]:
my_net_16 = glob(pathname = model_path + '\\mynet_16-*')

In [12]:
my_net_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16-5.keras']

In [13]:
mynet_16_dice = []
mynet_16_thin_dice = []
mynet_16_thick_dice = []

In [14]:
for path in my_net_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_16_dice.append(np.mean(full))
    mynet_16_thin_dice.append(np.mean(thin))
    mynet_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 30ms/step


In [15]:
np.mean(np.round(mynet_16_dice,3))

0.8320000000000001

In [16]:
np.mean(np.round(mynet_16_thin_dice,3))

0.3376

In [17]:
np.mean(np.round(mynet_16_thick_dice,3))

0.8287999999999999

# My-Net 8

In [18]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [19]:
my_net_8 = glob(pathname = model_path + '\\mynet_8-*')

In [20]:
my_net_8

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8-5.keras']

In [21]:
mynet_8_dice = []
mynet_8_thin_dice = []
mynet_8_thick_dice = []

In [22]:
for path in my_net_8:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_8_dice.append(np.mean(full))
    mynet_8_thin_dice.append(np.mean(thin))
    mynet_8_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 18ms/step


In [26]:
np.round(np.mean(mynet_8_dice),3)

0.797

In [27]:
np.round(np.mean(mynet_8_thin_dice),3)

0.282

In [28]:
np.round(np.mean(mynet_8_thick_dice),3)

0.793

In [29]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_180 (Conv2D)            (None, None, None,   224         ['input_10[0][0]']               
                                8)                                                                
                                                                                                  
 batch_normalization_45 (BatchN  (None, None, None,   32         ['conv2d_180[0][0]']             
 ormalization)                  8)                                                          

# Attention-UNet (16,32,64,128)

In [32]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [33]:
atten_unet_16 = glob(pathname = model_path + '\\atten_unet_16*')

In [34]:
atten_unet_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_16-5.keras']

In [35]:
atten_unet_16_dice = []
atten_unet_16_thin_dice = []
atten_unet_16_thick_dice = []

In [36]:
for path in atten_unet_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    atten_unet_16_dice.append(np.mean(full))
    atten_unet_16_thin_dice.append(np.mean(thin))
    atten_unet_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 25ms/step


In [37]:
np.round(np.mean(atten_unet_16_dice),3)

0.887

In [38]:
np.round(np.mean(atten_unet_16_thin_dice),3)

0.455

In [39]:
np.round(np.mean(atten_unet_16_thick_dice),3)

0.883

In [40]:
model.summary()

Model: "attunet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 attunet_down0_0 (Conv2D)       (None, None, None,   432         ['input_15[0][0]']               
                                16)                                                               
                                                                                                  
 attunet_down0_0_bn (BatchNorma  (None, None, None,   64         ['attunet_down0_0[0][0]']        
 lization)                      16)                                                   

# Attention-UNet (8,16,32,64)

In [41]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [42]:
atten_unet_8 = glob(pathname = model_path + '\\atten_unet_8*')

In [43]:
atten_unet_8

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\atten_unet_8-5.keras']

In [44]:
atten_unet_8_dice = []
atten_unet_8_thin_dice = []
atten_unet_8_thick_dice = []

In [45]:
for path in atten_unet_8:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    atten_unet_8_dice.append(np.mean(full))
    atten_unet_8_thin_dice.append(np.mean(thin))
    atten_unet_8_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 31ms/step


In [46]:
np.round(np.mean(atten_unet_8_dice),3)

0.882

In [47]:
np.round(np.mean(atten_unet_8_thin_dice),3)

0.431

In [48]:
np.round(np.mean(atten_unet_8_thick_dice),3)

0.879

In [52]:
model.summary()

Model: "attunet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 attunet_down0_0 (Conv2D)       (None, None, None,   216         ['input_20[0][0]']               
                                8)                                                                
                                                                                                  
 attunet_down0_0_bn (BatchNorma  (None, None, None,   32         ['attunet_down0_0[0][0]']        
 lization)                      8)                                                    

# Simple-Net 16

In [49]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [50]:
simple_net_16 = glob(pathname = model_path + '\\simplenet_16-*')

In [51]:
simple_net_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16-5.keras']

In [53]:
simplenet_16_dice = []
simplenet_16_thin_dice = []
simplenet_16_thick_dice = []

In [54]:
for path in simple_net_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    simplenet_16_dice.append(np.mean(full))
    simplenet_16_thin_dice.append(np.mean(thin))
    simplenet_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 30ms/step


In [55]:
np.round(np.mean(simplenet_16_dice),3)

0.83

In [56]:
np.round(np.mean(simplenet_16_thin_dice),3)

0.328

In [57]:
np.round(np.mean(simplenet_16_thick_dice),3)

0.827

In [58]:
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_272 (Conv2D)         (None, None, None, 16)    448       
                                                                 
 batch_normalization_62 (Bat  (None, None, None, 16)   64        
 chNormalization)                                                
                                                                 
 conv2d_273 (Conv2D)         (None, None, None, 16)    2320      
                                                                 
 activation_224 (Activation)  (None, None, None, 16)   0         
                                                                 
 conv2d_274 (Conv2D)         (None, None, None, 16)    2320      
                                                          

# Simple-Net 8

In [59]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [60]:
simple_net_8 = glob(pathname = model_path + '\\simplenet_8-*')

In [61]:
simple_net_8

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8-5.keras']

In [62]:
simplenet_8_dice = []
simplenet_8_thin_dice = []
simplenet_8_thick_dice = []

In [63]:
for path in simple_net_8:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    simplenet_8_dice.append(np.mean(full))
    simplenet_8_thin_dice.append(np.mean(thin))
    simplenet_8_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 23ms/step


In [64]:
np.round(np.mean(simplenet_8_dice),3)

0.796

In [65]:
np.round(np.mean(simplenet_8_thin_dice),3)

0.254

In [66]:
np.round(np.mean(simplenet_8_thick_dice),3)

0.794

In [67]:
model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_362 (Conv2D)         (None, None, None, 8)     224       
                                                                 
 batch_normalization_77 (Bat  (None, None, None, 8)    32        
 chNormalization)                                                
                                                                 
 conv2d_363 (Conv2D)         (None, None, None, 8)     584       
                                                                 
 activation_304 (Activation)  (None, None, None, 8)    0         
                                                                 
 conv2d_364 (Conv2D)         (None, None, None, 8)     584       
                                                          

# DeepLabV3+

In [68]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [69]:
deeplab = glob(pathname = model_path + '\\deeplab*')

In [70]:
deeplab

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\deeplab-5.keras']

In [71]:
deeplab_dice = []
deeplab_thin_dice = []
deeplab_thick_dice = []

In [72]:
for path in deeplab:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_metric ,'dice_coeff_loss': dice_loss,"ConvBlock":ConvBlock})
    full,thin,thick = evaluate(model,512,None,None,None)
    deeplab_dice.append(np.mean(full))
    deeplab_thin_dice.append(np.mean(thin))
    deeplab_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 30ms/step


In [73]:
np.round(np.mean(deeplab_dice),3)

0.868

In [74]:
np.round(np.mean(deeplab_thin_dice),3)

0.347

In [75]:
np.round(np.mean(deeplab_thick_dice),3)

0.867

In [76]:
model.summary()

Model: "DeepLabV3-Plus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['InputLayer[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                     

# My-Net (32 filters)

In [77]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [78]:
my_net_32 = glob(pathname = model_path + '\\mynet_32-*')

In [79]:
my_net_32

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32-5.keras']

In [80]:
mynet_32_dice = []
mynet_32_thin_dice = []
mynet_32_thick_dice = []

In [81]:
for path in my_net_32:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_32_dice.append(np.mean(full))
    mynet_32_thin_dice.append(np.mean(thin))
    mynet_32_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 30ms/step


In [82]:
np.round(np.mean(mynet_32_dice),3)

0.848

In [83]:
np.round(np.mean(mynet_32_thin_dice),3)

0.386

In [84]:
np.round(np.mean(mynet_32_thick_dice),3)

0.843

In [85]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_80 (Conv2D)             (None, None, None,   896         ['input_5[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_20 (BatchN  (None, None, None,   128        ['conv2d_80[0][0]']              
 ormalization)                  32)                                                         

# Simple-Net 32

In [86]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [87]:
simple_net_32 = glob(pathname = model_path + '\\simplenet_32-*')

In [88]:
simple_net_32

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_32-5.keras']

In [89]:
simplenet_32_dice = []
simplenet_32_thin_dice = []
simplenet_32_thick_dice = []

In [90]:
for path in simple_net_32:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    simplenet_32_dice.append(np.mean(full))
    simplenet_32_thin_dice.append(np.mean(thin))
    simplenet_32_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 30ms/step


In [91]:
np.round(np.mean(simplenet_32_dice),3)

0.848

In [92]:
np.round(np.mean(simplenet_32_thin_dice),3)

0.38

In [93]:
np.round(np.mean(simplenet_32_thick_dice),3)

0.844

In [94]:
model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_479 (Conv2D)         (None, None, None, 32)    896       
                                                                 
 batch_normalization_89 (Bat  (None, None, None, 32)   128       
 chNormalization)                                                
                                                                 
 conv2d_480 (Conv2D)         (None, None, None, 32)    9248      
                                                                 
 activation_416 (Activation)  (None, None, None, 32)   0         
                                                                 
 conv2d_481 (Conv2D)         (None, None, None, 32)    9248      
                                                          

# Pocket-Net 16

In [99]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [100]:
pocket_net_16 = glob(pathname = model_path + '\\pocketnet_16-*')

In [101]:
pocket_net_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\pocketnet_16-5.keras']

In [102]:
pocketnet_16_dice = []
pocketnet_16_thin_dice = []
pocketnet_16_thick_dice = []

In [103]:
for path in pocket_net_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    pocketnet_16_dice.append(np.mean(full))
    pocketnet_16_thin_dice.append(np.mean(thin))
    pocketnet_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 21ms/step


In [104]:
np.round(np.mean(pocketnet_16_dice),3)

0.869

In [105]:
np.round(np.mean(pocketnet_16_thin_dice),3)

0.39

In [106]:
np.round(np.mean(pocketnet_16_thick_dice),3)

0.865

In [107]:
model.summary()

Model: "attunet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 attunet_down0_0 (Conv2D)       (None, None, None,   432         ['input_5[0][0]']                
                                16)                                                               
                                                                                                  
 attunet_down0_0_bn (BatchNorma  (None, None, None,   64         ['attunet_down0_0[0][0]']        
 lization)                      16)                                                   

# Restype-Net 16 (Wave Net)

In [108]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [109]:
restype_net_16 = glob(pathname = model_path + '\\restype_16-*')

In [110]:
restype_net_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\restype_16-5.keras']

In [111]:
restypenet_16_dice = []
restypenet_16_thin_dice = []
restypenet_16_thick_dice = []

In [112]:
for path in restype_net_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    restypenet_16_dice.append(np.mean(full))
    restypenet_16_thin_dice.append(np.mean(thin))
    restypenet_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 41ms/step


In [113]:
np.round(np.mean(restypenet_16_dice),3)

0.879

In [114]:
np.round(np.mean(restypenet_16_thin_dice),3)

0.432

In [115]:
np.round(np.mean(restypenet_16_thick_dice),3)

0.875

In [116]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_184 (Conv2D)            (None, None, None,   448         ['input_10[0][0]']               
                                16)                                                               
                                                                                                  
 batch_normalization_52 (BatchN  (None, None, None,   64         ['conv2d_184[0][0]']             
 ormalization)                  16)                                                         

# My-Net 16 (16 blocks)

In [127]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [128]:
my_net_16_16b = glob(pathname = model_path + '\\mynet_16_*')

In [129]:
my_net_16_16b

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_16b-5.keras']

In [130]:
mynet_16_16b_dice = []
mynet_16_16b_thin_dice = []
mynet_16_16b_thick_dice = []

In [131]:
for path in my_net_16_16b:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_16_16b_dice.append(np.mean(full))
    mynet_16_16b_thin_dice.append(np.mean(thin))
    mynet_16_16b_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 44ms/step


In [132]:
np.round(np.mean(mynet_16_16b_dice),3)

0.84

In [133]:
np.round(np.mean(mynet_16_16b_thin_dice),3)

0.345

In [134]:
np.round(np.mean(mynet_16_16b_thick_dice),3)

0.837

# My-Net 8 (16 blocks)

In [144]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [145]:
my_net_8_16b = glob(pathname = model_path + '\\mynet_8_*')

In [146]:
my_net_8_16b

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_16b-5.keras']

In [147]:
mynet_8_16b_dice = []
mynet_8_16b_thin_dice = []
mynet_8_16b_thick_dice = []

In [148]:
for path in my_net_8_16b:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_8_16b_dice.append(np.mean(full))
    mynet_8_16b_thin_dice.append(np.mean(thin))
    mynet_8_16b_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 47ms/step


In [149]:
np.round(np.mean(mynet_8_16b_dice),3)

0.769

In [150]:
np.round(np.mean(mynet_8_16b_thin_dice),3)

0.26

In [151]:
np.round(np.mean(mynet_8_16b_thick_dice),3)

0.762

In [152]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_370 (Conv2D)            (None, None, None,   224         ['input_11[0][0]']               
                                8)                                                                
                                                                                                  
 batch_normalization_70 (BatchN  (None, None, None,   32         ['conv2d_370[0][0]']             
 ormalization)                  8)                                                         

# Simple-Net 16 (16 blocks)

In [153]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [154]:
simple_net_16_16b = glob(pathname = model_path + '\\simplenet_16_*')

In [155]:
simple_net_16_16b

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_16_16b-5.keras']

In [156]:
simplenet_16_16b_dice = []
simplenet_16_16b_thin_dice = []
simplenet_16_16b_thick_dice = []

In [157]:
for path in simple_net_16_16b:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    simplenet_16_16b_dice.append(np.mean(full))
    simplenet_16_16b_thin_dice.append(np.mean(thin))
    simplenet_16_16b_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 42ms/step


In [158]:
np.round(np.mean(simplenet_16_16b_dice),3)

0.825

In [159]:
np.round(np.mean(simplenet_16_16b_thin_dice),3)

0.314

In [160]:
np.round(np.mean(simplenet_16_16b_thick_dice),3)

0.825

In [161]:
model.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_633 (Conv2D)         (None, None, None, 16)    448       
                                                                 
 batch_normalization_108 (Ba  (None, None, None, 16)   64        
 tchNormalization)                                               
                                                                 
 conv2d_634 (Conv2D)         (None, None, None, 16)    2320      
                                                                 
 activation_560 (Activation)  (None, None, None, 16)   0         
                                                                 
 conv2d_635 (Conv2D)         (None, None, None, 16)    2320      
                                                          

# Simple-Net 8 (16 blocks)

In [162]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [163]:
simple_net_8_16b = glob(pathname = model_path + '\\simplenet_8_*')

In [164]:
simple_net_8_16b

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\simplenet_8_16b-5.keras']

In [165]:
simplenet_8_16b_dice = []
simplenet_8_16b_thin_dice = []
simplenet_8_16b_thick_dice = []

In [166]:
for path in simple_net_8_16b:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    simplenet_8_16b_dice.append(np.mean(full))
    simplenet_8_16b_thin_dice.append(np.mean(thin))
    simplenet_8_16b_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 50ms/step


In [167]:
np.round(np.mean(simplenet_8_16b_dice),3)

0.785

In [168]:
np.round(np.mean(simplenet_8_16b_thin_dice),3)

0.267

In [169]:
np.round(np.mean(simplenet_8_16b_thick_dice),3)

0.78

In [170]:
model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_803 (Conv2D)         (None, None, None, 8)     224       
                                                                 
 batch_normalization_128 (Ba  (None, None, None, 8)    32        
 tchNormalization)                                               
                                                                 
 conv2d_804 (Conv2D)         (None, None, None, 8)     584       
                                                                 
 activation_720 (Activation)  (None, None, None, 8)    0         
                                                                 
 conv2d_805 (Conv2D)         (None, None, None, 8)     584       
                                                          

# My-Net 16 - Concat

In [171]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [172]:
my_net_16_concat = glob(pathname = model_path + '\\mynet_16_concat*')

In [173]:
my_net_16_concat

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_concat-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_concat-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_concat-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_concat-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_16_concat-5.keras']

In [174]:
mynet_16_concat_dice = []
mynet_16_concat_thin_dice = []
mynet_16_concat_thick_dice = []

In [175]:
for path in my_net_16_concat:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_16_concat_dice.append(np.mean(full))
    mynet_16_concat_thin_dice.append(np.mean(thin))
    mynet_16_concat_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 55ms/step


In [176]:
np.round(np.mean(mynet_16_concat_dice),3)

0.841

In [177]:
np.round(np.mean(mynet_16_concat_thin_dice),3)

0.354

In [178]:
np.round(np.mean(mynet_16_concat_thick_dice),3)

0.836

# My-Net 32 - Concat

In [179]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [180]:
my_net_32_concat = glob(pathname = model_path + '\\mynet_32_concat*')

In [181]:
my_net_32_concat

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32_concat-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32_concat-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32_concat-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32_concat-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_32_concat-5.keras']

In [182]:
mynet_32_concat_dice = []
mynet_32_concat_thin_dice = []
mynet_32_concat_thick_dice = []

In [183]:
for path in my_net_32_concat:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_32_concat_dice.append(np.mean(full))
    mynet_32_concat_thin_dice.append(np.mean(thin))
    mynet_32_concat_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 100ms/step


In [184]:
np.round(np.mean(mynet_32_concat_dice),3)

0.842

In [185]:
np.round(np.mean(mynet_32_concat_thin_dice),3)

0.371

In [186]:
np.round(np.mean(mynet_32_concat_thick_dice),3)

0.837

# My-Net 8 - Concat

In [187]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [188]:
my_net_8_concat = glob(pathname = model_path + '\\mynet_8_concat*')

In [189]:
my_net_8_concat

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_concat-1.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_concat-2.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_concat-3.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_concat-4.keras',
 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\mynet_8_concat-5.keras']

In [190]:
mynet_8_concat_dice = []
mynet_8_concat_thin_dice = []
mynet_8_concat_thick_dice = []

In [191]:
for path in my_net_8_concat:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    mynet_8_concat_dice.append(np.mean(full))
    mynet_8_concat_thin_dice.append(np.mean(thin))
    mynet_8_concat_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 58ms/step


In [192]:
np.round(np.mean(mynet_8_concat_dice),3)

0.809

In [193]:
np.round(np.mean(mynet_8_concat_thin_dice),3)

0.281

In [194]:
np.round(np.mean(mynet_8_concat_thick_dice),3)

0.807

# Separable 16

In [29]:
model_path = 'G:\\Harsh_Birva_minor_Proj\\new_retina_models_5'

In [30]:
separable_16 = glob(pathname = model_path + '\\separable_upsample_16*')

In [31]:
separable_16

['G:\\Harsh_Birva_minor_Proj\\new_retina_models_5\\separable_upsample_16.keras']

In [32]:
separable_16_dice = []
separable_16_thin_dice = []
separable_16_thick_dice = []

In [33]:
for path in separable_16:
    model = tf.keras.models.load_model(path, custom_objects={'dice_coeff': dice_coeff ,'dice_coeff_loss': dice_coeff_loss})
    full,thin,thick = evaluate(model,512,None,None,None)
    separable_16_dice.append(np.mean(full))
    separable_16_thin_dice.append(np.mean(thin))
    separable_16_thick_dice.append(np.mean(thick))

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 23ms/step


In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 separable_conv2d (SeparableCon  (None, None, None,   91         ['input_1[0][0]']                
 v2D)                           16)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   64         ['separable_conv2d[0][0]']       
 alization)                     16)                                                           

                                                                                                  
 separable_conv2d_14 (Separable  (None, None, None,   9408       ['batch_normalization_5[0][0]']  
 Conv2D)                        64)                                                               
                                                                                                  
 up_sampling2d_1 (UpSampling2D)  (None, None, None,   0          ['separable_conv2d_14[0][0]']    
                                64)                                                               
                                                                                                  
 add_1 (Add)                    (None, None, None,   0           ['up_sampling2d_1[0][0]',        
                                64)                               'batch_normalization_3[0][0]']  
                                                                                                  
 separable

In [35]:
np.round(np.mean(separable_16_dice),3)

0.87

In [36]:
np.round(np.mean(separable_16_thin_dice),3)

0.425

In [37]:
np.round(np.mean(separable_16_thick_dice),3)

0.866